In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

# Define the API endpoint and parameters
url = 'https://api.binance.com/api/v3/klines'
params = {
    'symbol': 'ATOMUSDT',
    'interval': '15m',
    'limit': 1000  # maximum value is 1000
}

# Send a GET request to the API endpoint
response = requests.get(url, params=params)

# Parse the JSON response into a Pandas DataFrame
data = pd.read_json(response.text)
data.columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

# Convert the timestamp to a readable format
data['Open time'] = pd.to_datetime(data['Open time'], unit='ms')

# Remove unnecessary columns
data = data[['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']]

# Set the index to the timestamp column
data.set_index('Open time', inplace=True)

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split the data into training and test sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size, :]
test_data = scaled_data[train_size:, :]

# Define the window size for the LSTM model
window_size = 60

# Define the training data and labels
X_train = []
y_train = []

for i in range(window_size, len(train_data)):
    X_train.append(train_data[i-window_size:i, :])
    y_train.append(train_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

num_features = 5 

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(window_size, num_features)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=5)

# Prepare the test data
X_test = []
y_test = []

for i in range(window_size, len(test_data)):
    X_test.append(test_data[i-window_size:i, :])
    y_test.append(test_data[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

# Make predictions on the test data
predictions = model.predict(X_test)

# Combine the predicted values with the test data
combined = np.concatenate((predictions, X_test[:, :, 1:]), axis=0)

# Scale the combined data back to their original values
scaled = scaler.inverse_transform(combined.reshape(-1, num_features))
predictions = scaled[:, 0]

# Get the actual closing prices from the test data
y_test = scaler.inverse_transform(y_test.reshape(-1, num_features))[:, 0]

# Evaluate the model using mean squared error (MSE)
mse = np.mean(np.square(predictions - y_test))
print('MSE:', mse)

# Plot the predictions against the actual values
plt.plot(data.index[train_size+window_size:], y_test, color='red', label='Actual price')
plt.plot(data.index[train_size+window_size:], predictions, color='blue', label='Predicted price')
plt.legend()
plt.show()


2023-04-08 19:16:36.540974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-08 19:16:36.543133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-08 19:16:36.544501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5


2023-04-08 19:16:37.033799: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-08 19:16:37.036114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-08 19:16:37.037629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 [==============================] - 4s 53ms/step - loss: 0.0536
Epoch 2/5
12/12 [==============================] - 1s 56ms/step - loss: 0.0178
Epoch 3/5
12/12 [==============================] - 1s 78ms/step - loss: 0.0115
Epoch 4/5
12/12 [==============================] - 1s 60ms/step - loss: 0.0096
Epoch 5/5
12/12 [==============================] - 1s 53ms/step - loss: 0.0083


2023-04-08 19:16:45.311901: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-08 19:16:45.313900: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-08 19:16:45.315250: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5/5 [==============================] - 1s 11ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

: 